# Pull Weather Data from Dark Sky

In [1]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import seaborn as sbn
import matplotlib.pyplot as plt
import statsmodels.api as sm
from io import StringIO
import requests
import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor

from darksky import forecast
from datetime import datetime as dt
from datetime import date as date
#from date import timedelta
import numpy as np
import pandas as pd

from datetime import timedelta

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [2]:
# This is a public data downloaded from the web to map county to its Latitude and Longitude
df_Countyyieldmap = pd.read_csv('County_to_Lat_Long_Map_YEILD.csv')
df_Countyyieldmap.head()

,USPS,County_Name,INTPTLAT,INTPTLONG
0,CA,Fresno,36.761006,-119.655019
1,CA,Kern,35.346629,-118.729506
2,CA,Kings,36.072478,-119.815530
3,CA,Merced,37.194806,-120.722802
4,CA,San Joaquin,37.934982,-121.272244


In [3]:
# Generate API Keys from DarkSky
# URL: https://darksky.net/dev
df_APIKey = pd.read_csv('API_Keys.csv')
df_APIKey.head()

,API Keys
0,3c8d97078b78b66d3d903268cefc41cd
1,fe67024d47920837be2053b1f7f5493d
2,7f8ab5cfa93335d63f3da8b07a05b106
3,68b511afb8c8b22c6769892c500315f2
4,5ab97829363d4a313f1fd8c8ce59c38d


## Pull 2019 Weather Data between Feb-May for all the eight counties

In [5]:
Temperature = []
PrecipIntensity = []
PrecipProbability = []
Pressure = []
Sunlight = []
DewPoint = []
Date = []
countyName = []
stateName = []

def get_WeatherData(list):
#list = [[1999,2000,2001,2002],[2003,2004,2005,2006],[2007,2008,2009,2010],[2011,2012,2013,2014],[2015,2016,2017,2018]]
    apiKeys = df_APIKey['API Keys'].values.tolist()
    i=0;
    for index, row in df_Countyyieldmap.iterrows():
      for past_years in list:
        row
        if(i >= len(apiKeys)):
          print("error")
          print(i)
        #print (row)
        #print (row['INTPTLONG'])
        COUNTY = apiKeys[i], row['INTPTLAT'], row['INTPTLONG']
        print (i)
        i=i+1
        print(COUNTY)
        for year_id in past_years:
          print(year_id)
          start_dt = date(year_id, 2, 1)
          end_dt = date(year_id, 5, 31)
          for dt1 in daterange(start_dt, end_dt):
            date_variable = dt1.isoformat()
            t = dt(dt1.year, dt1.month, dt1.day).isoformat() #y, m, d
            County = forecast(*COUNTY, time=t)
            if hasattr(County, 'hourly'):
              County.hourly
              temperature = []
              precipIntensity = []
              precipProbability = []
              pressure = []
              sunlight = []
              dewPoint = []

              for j in range(0,len(County.hourly)):
                if hasattr(County.hourly[j], 'temperature'):
                  temperature.append(County.hourly[j].temperature)
                if hasattr(County.hourly[j], 'precipIntensity'):
                  precipIntensity.append(County.hourly[j].precipIntensity)
                if hasattr(County.hourly[j], 'precipProbability'):
                  precipProbability.append(County.hourly[j].precipProbability)
                if hasattr(County.hourly[j], 'pressure'):
                  pressure.append(County.hourly[j].pressure)
                if hasattr(County.hourly[j], 'uvIndex'):
                  sunlight.append(County.hourly[j].uvIndex)
                if hasattr(County.hourly[j], 'dewPoint'):
                  dewPoint.append(County.hourly[j].dewPoint)

              mean_temperature = np.mean(temperature)
              mean_precipIntensity = np.mean(precipIntensity)
              mean_precipProbability = np.mean(precipProbability)
              mean_pressure = np.mean(pressure)
              mean_sunlight = np.mean(sunlight)
              mean_dewPoint = np.mean(dewPoint)

              Date.append(date_variable)
              Temperature.append(mean_temperature)
              PrecipIntensity.append(mean_precipIntensity) 
              PrecipProbability.append(mean_precipProbability) 
              Pressure.append(mean_pressure) 
              Sunlight.append(mean_sunlight)
              DewPoint.append(mean_dewPoint) 
              countyName.append(row['County_Name'])
              stateName.append(row['USPS'])
              d = {'Date':Date, 'Temperature':Temperature, 'PrecipIntensity':PrecipIntensity, 'PrecipProbability':PrecipProbability, 'Pressure':Pressure, 'Sunlight':Sunlight, 'DewPoint':DewPoint, 'County':countyName, 'State':stateName}
              df1 = pd.DataFrame(d)
    return df1
              

list = [[2019]]
df = get_WeatherData(list)
df.to_csv('2019WeatherData.csv')

0
('3c8d97078b78b66d3d903268cefc41cd', 36.761006, -119.655019)
2019
1
('fe67024d47920837be2053b1f7f5493d', 35.346629, -118.729506)
2019
2
('7f8ab5cfa93335d63f3da8b07a05b106', 36.072478000000004, -119.81553000000001)
2019
3
('68b511afb8c8b22c6769892c500315f2', 37.194806, -120.722802)
2019
4
('5ab97829363d4a313f1fd8c8ce59c38d', 37.934982, -121.272244)
2019
5
('ae4016b78d182cce183484e09e8bcc26', 38.267226, -121.939594)
2019
6
('fe67024d47920837be2053b1f7f5493d', 37.562315999999996, -121.002831)
2019
7
('a5c35d3845c30819c69e60f36091b0eb', 38.679595, -121.902441)
2019


In [13]:
df = pd.read_csv('2019WeatherData.csv')
df = df.drop(columns = "Unnamed: 0")
df.head()

,Date,Temperature,PrecipIntensity,PrecipProbability,Pressure,Sunlight,DewPoint,County,State
0,2019-02-01,54.899583,0.000554,0.058750,1012.112500,0.666667,48.088750,Fresno,CA
1,2019-02-02,55.204583,0.042196,0.684583,1003.739167,0.583333,48.705000,Fresno,CA
2,2019-02-03,53.631667,0.010879,0.372917,1007.328333,0.583333,46.943333,Fresno,CA
3,2019-02-04,50.668750,0.018958,0.430833,1005.903333,0.625000,43.654583,Fresno,CA
4,2019-02-05,43.509565,0.004617,0.214348,1012.170000,0.652174,34.397826,Fresno,CA


## Pull Historic Weather data(2015-2018) for all the counties from Feb - Jun

In [ ]:
Temperature = []
PrecipIntensity = []
PrecipProbability = []
Pressure = []
Sunlight = []
DewPoint = []
Date = []
countyName = []
stateName = []

def get_WeatherData_Historic(list):
    apiKeys = df_APIKey['API Keys'].values.tolist()
    i=0;
    for index, row in df_Countyyieldmap.iterrows():
      for past_years in list:
        row
        if(i >= len(apiKeys)):
          print("error")
          print(i)
        COUNTY = apiKeys[i], row['INTPTLAT'], row['INTPTLONG']
        i=i+1
        for year_id in past_years:
          print(year_id)
          start_dt = date(year_id, 2, 1)
          end_dt = date(year_id, 5, 31)
          for dt1 in daterange(start_dt, end_dt):
            date_variable = dt1.isoformat()
            t = dt(dt1.year, dt1.month, dt1.day).isoformat() #y, m, d
            County = forecast(*COUNTY, time=t)
            if hasattr(County, 'hourly'):
              County.hourly
              temperature = []
              precipIntensity = []
              precipProbability = []
              pressure = []
              sunlight = []
              dewPoint = []

              for j in range(0,len(County.hourly)):
                if hasattr(County.hourly[j], 'temperature'):
                  temperature.append(County.hourly[j].temperature)
                if hasattr(County.hourly[j], 'precipIntensity'):
                  precipIntensity.append(County.hourly[j].precipIntensity)
                if hasattr(County.hourly[j], 'precipProbability'):
                  precipProbability.append(County.hourly[j].precipProbability)
                if hasattr(County.hourly[j], 'pressure'):
                  pressure.append(County.hourly[j].pressure)
                if hasattr(County.hourly[j], 'uvIndex'):
                  sunlight.append(County.hourly[j].uvIndex)
                if hasattr(County.hourly[j], 'dewPoint'):
                  dewPoint.append(County.hourly[j].dewPoint)

              mean_temperature = np.mean(temperature)
              mean_precipIntensity = np.mean(precipIntensity)
              mean_precipProbability = np.mean(precipProbability)
              mean_pressure = np.mean(pressure)
              mean_sunlight = np.mean(sunlight)
              mean_dewPoint = np.mean(dewPoint)

              Date.append(date_variable)
              Temperature.append(mean_temperature)
              PrecipIntensity.append(mean_precipIntensity) 
              PrecipProbability.append(mean_precipProbability) 
              Pressure.append(mean_pressure) 
              Sunlight.append(mean_sunlight)
              DewPoint.append(mean_dewPoint) 
              countyName.append(row['County_Name'])
              stateName.append(row['USPS'])
              d = {'Date':Date, 'Temperature':Temperature, 'PrecipIntensity':PrecipIntensity, 'PrecipProbability':PrecipProbability, 'Pressure':Pressure, 'Sunlight':Sunlight, 'DewPoint':DewPoint, 'County':countyName, 'State':stateName}
              df1 = pd.DataFrame(d)
    return df1
              

list = [[2015,2016,2017,2018]]
dfHistoric = get_WeatherData_Historic(list)
dfHistoric.to_csv('HistoricWeatherData.csv')

In [16]:
df_Historic = pd.read_csv('HistoricWeatherData.csv')
df_Historic.head()

,Date,Temperature,PrecipIntensity,PrecipProbability,Pressure,Sunlight,DewPoint,County,State
0,2/1/15,52.029583,0.0,0.0,1020.519583,0.541667,43.455000,Yolo,CA
1,2/2/15,53.700833,0.0,0.0,1021.007917,0.625000,45.670833,Yolo,CA
2,2/3/15,55.718333,0.0,0.0,1019.744167,0.625000,48.891250,Yolo,CA
3,2/4/15,54.168333,0.0,0.0,1017.838333,0.666667,49.035833,Yolo,CA
4,2/5/15,53.848750,0.0,0.0,1014.792083,0.708333,48.928750,Yolo,CA


## Merge 2019 Weather Data with Historic Data

In [19]:
dfWeather = df.append(df_Historic)
dfWeather.head()

,Date,Temperature,PrecipIntensity,PrecipProbability,Pressure,Sunlight,DewPoint,County,State
0,2019-02-01,54.899583,0.000554,0.058750,1012.112500,0.666667,48.088750,Fresno,CA
1,2019-02-02,55.204583,0.042196,0.684583,1003.739167,0.583333,48.705000,Fresno,CA
2,2019-02-03,53.631667,0.010879,0.372917,1007.328333,0.583333,46.943333,Fresno,CA
3,2019-02-04,50.668750,0.018958,0.430833,1005.903333,0.625000,43.654583,Fresno,CA
4,2019-02-05,43.509565,0.004617,0.214348,1012.170000,0.652174,34.397826,Fresno,CA


In [24]:
df.to_csv('WeatherData.csv')